# Calculate median weekly trends in snowlines for all sites

In [ ]:
import os
import glob
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
import sys
from scipy.stats import iqr
from shapely import wkt
import seaborn as sns
import contextily as ctx
import numpy as np

## Define paths in directory, import functions

In [ ]:
base_path = '/Users/raineyaberle/Research/PhD/snow_cover_mapping/snow-cover-mapping-application/'
figures_out_path = os.path.join(base_path, 'figures')
sys.path.append(os.path.join(base_path, 'functions'))
import model_analyze_utils as f

scm_path = '/Users/raineyaberle/Research/PhD/snow_cover_mapping/'

## Load compiled glacier boundaries (AOIs) and snowlines

In [ ]:
# -----Load AOIs
aois_fn = os.path.join(scm_path, 'compiled_data', 'all_aois_climate_cluster.shp')
aois = gpd.read_file(aois_fn)
print('All glacier boundaries loaded from file.')

# -----Load compiled snow cover stats
scs_fn = os.path.join(scm_path, 'compiled_data', 'all_snow_cover_stats.csv')
scs = pd.read_csv(scs_fn)
print('All snow cover stats loaded from file.')

## Calculate weekly median trends for each site

In [ ]:
scs_medians_fn = os.path.join(scm_path, 'results', 'all_snow_cover_stats_weekly_median_trends.csv') 
if not os.path.exists(scs_medians_fn):
    # add week-of-year (WOY) column to snow cover stats
    scs['datetime'] = pd.to_datetime(scs['datetime'], format='mixed')
    scs['WOY'] = scs['datetime'].dt.isocalendar().week
    # determine columns to calculate weekly stats
    columns = ['AAR', 'snowline_elevs_median_m', 'SCA_m2', 'ELA_from_AAR_m']
    scs_medians = pd.DataFrame()
    for rgi_id in tqdm(scs['RGIId'].drop_duplicates().values):
        # subset snow cover stats to site
        scs_site = scs.loc[scs['RGIId']==rgi_id]
        # calculate weekly quartile trends
        q1 = scs_site[['WOY'] + columns].groupby(by='WOY').quantile(0.25)
        q1.columns = [x + '_P25' for x in q1.columns]
        q2 = scs_site[['WOY'] + columns].groupby(by='WOY').quantile(0.5)
        q2.columns = [x + '_P50' for x in q2.columns]
        q3 = scs_site[['WOY'] + columns].groupby(by='WOY').quantile(0.75)
        q3.columns = [x + '_P75' for x in q3.columns]
        qs = pd.merge(q1, pd.merge(q2, q3, on='WOY'), on='WOY')
        qs = qs.reindex(sorted(qs.columns), axis=1)
        qs['WOY'] = qs.index
        qs['RGIId'] = rgi_id
        # concatenate to medians dataframe
        scs_medians = pd.concat([scs_medians, qs])
    # save to file
    scs_medians.to_csv(scs_medians_fn, index=False)
    print('Median weekly snow trends saved to file: ', scs_medians_fn)
        
else:
    scs_medians = pd.read_csv(scs_medians_fn)
    print('Median weekly snow cover trends loaded from file.')
    
scs_medians
        

In [ ]:
# -----Compile RGI characteristics and minimum snow cover median statistics
min_snow_cover_stats_fn = os.path.join(scm_path, 'results', 'min_snow_cover_stats.csv') 
# check if exists in directory
if not os.path.exists(min_snow_cover_stats_fn):
    # initialize dataframe for RGI stats and minimum snow cover statts
    min_snow_cover_stats = pd.DataFrame()
    
    # iterate over site names in median snow cover stats dataframe
    for rgi_id in tqdm(sorted(scs_medians['RGIId'].drop_duplicates().values)):
        # grab AOI for site
        aoi_site = aois.loc[aois['RGIId']==rgi_id, :]
        # grab median snowline stats for site
        scs_medians_site = scs_medians.loc[scs_medians['RGIId']==rgi_id, :]
        # calculate min median stats
        median_columns = [x for x in scs_medians.columns if 'P50' in x]
        for column in median_columns:
            if (column=='ELA_from_AAR_m_P50') or (column=='snowline_elevs_median_m_P50'):
                aoi_site[column+'_max'] = scs_medians_site[column].max()
            else:
                aoi_site[column+'_min'] = scs_medians_site[column].min()
        # concatenate to full dataframe
        min_snow_cover_stats = pd.concat([min_snow_cover_stats, aoi_site])

    # add subregion names and colors
    min_snow_cover_stats[['Subregion', 'color']] = '', ''
    min_snow_cover_stats[['O1Region', 'O2Region']] = min_snow_cover_stats[['O1Region', 'O2Region']].astype(int)
    for o1, o2 in min_snow_cover_stats[['O1Region', 'O2Region']].drop_duplicates().values:
        min_snow_cover_stats.loc[(min_snow_cover_stats['O1Region']==o1) 
                                 & (min_snow_cover_stats['O2Region']==o2), ['Subregion', 'color']] = f.determine_subregion_name_color(o1, o2)
    # save to file
    min_snow_cover_stats.to_csv(min_snow_cover_stats_fn, index=False)
    print('Minimum median snow cover stats saved to file: ', min_snow_cover_stats_fn)
        
else:
    # load from file
    min_snow_cover_stats = pd.read_csv(min_snow_cover_stats_fn)
    min_snow_cover_stats['geometry'] = min_snow_cover_stats['geometry'].apply(wkt.loads)
    print('Minimum median snow cover stats loaded from file.')

# reformat as GeoDataFrame
min_snow_cover_stats = gpd.GeoDataFrame(min_snow_cover_stats, crs='EPSG:4326')
min_snow_cover_stats

In [ ]:
fig, ax = plt.subplots(2, 1, figsize=(10, 12))
sns.histplot(data=min_snow_cover_stats, x='AAR_P50_min', multiple='stack', hue='Subregion', 
             bins=np.arange(0, 1, step=0.1), ax=ax[0])
sns.histplot(data=min_snow_cover_stats, x='AAR_P50_min', multiple='stack', hue='clustName', 
             bins=np.arange(0, 1, step=0.1), ax=ax[1])
plt.show()

In [ ]:
# Print stats
min_snow_cover_stats.groupby(by=['Subregion', 'clustName'])['AAR_P50_min'].median()